In [1]:
!pip install datasets -qq
!pip install transformers -qq
!pip install accelerate -qq
!pip install python-Levenshtein -qq
!pip install fuzzywuzzy -qq

In [2]:
from typing import Dict, List, Union
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [3]:
MessageHint = Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]

In [4]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> MessageHint:
    """

    Функция добавляет к текстам prompt с инструкцией для LLM.

    """
    assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'
    categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\nТекст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content':
                'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\nТекст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content':
                    'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}

In [5]:
test_message = """Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).
Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся с
неразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенно
различными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».
В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием мира.
""".replace('\n', '')
test_categories = ['философия', 'математика', 'физика']
prepare_message_for_llm(test_message, test_categories)

{'message_for_llm': [{'role': 'system',
   'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'},
  {'role': 'user',
   'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: философия, математика и физика.\nТекст: Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся снеразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенноразличными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием мир

# Qwen2

Technical report - https://arxiv.org/pdf/2407.10671  

https://qwenlm.github.io/blog/qwen2/





* Размеры от 0.5B до 72B.

* 29 языков, включая русский.

* Поддержка контекстного окна до 128K токенов у Qwen2-7B-Instruct и Qwen2-72B-Instruct.

* Использование Grouped Query Attention (GQA).




In [6]:
llm_pipeline = pipeline(model='Qwen/Qwen2-1.5B-Instruct', device_map='auto', torch_dtype='auto')

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

### Выгрузим данные

https://huggingface.co/datasets/Davlan/sib200/viewer/rus_Cyrl

In [12]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [13]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


### Добавим prompt

In [14]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], list_of_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], list_of_categories))
print(validation_set_for_llm)
print(validation_set_for_llm['message_for_llm'][0])

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})
[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.\nВаш ответ: ', 'role': 'user'}]


### Функция для просмотра количества полученных и реальных категорий

In [15]:
from collections import Counter
import pandas as pd


def get_predictions_stat(y_pred, y_true):
    # Получаем категории из предсказанных и истинных меток
    predicted_categories = y_pred
    true_categories = y_true

    # Считаем количество предсказаний для каждой категории
    predictions_count = Counter(predicted_categories)
    true_count = Counter(true_categories)

    # Получаем уникальные категории из предсказанных и истинных меток
    all_categories = set(predicted_categories).union(set(true_categories))
    results = []

    for category in all_categories:
        count = predictions_count.get(category, 0)  # Количество предсказанных для этой категории
        true_count_val = true_count.get(category, 0)  # Количество истинных для этой категории
        correct_count = sum(1 for true, pred in zip(true_categories, predicted_categories) if true == pred and pred == category)
        accuracy = (correct_count / count) * 100 if count > 0 else 0  # Процент правильных предсказаний

        results.append({
            'Category': category,
            'Predicted Count': count,
            'True Count': true_count_val,
            'Correct Count': correct_count,
            'Accuracy (%)': accuracy
        })

    # Преобразуем результаты в DataFrame для удобного отображения
    results_df = pd.DataFrame(results)
    print("All predicted and real categories:")
    print(list(all_categories))
    return results_df

### Сгенерируем ответы модели

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
def get_predictions(model, dataset, normalize=False, cut_ss=False):
    def process_text(text):
        # Удаляем окончательные буквы 's' и приводим к нижнему регистру
        while text.endswith('s'):
            text = text[:-1]
        return text.lower()

    # Получаем предсказания от модели
    pred = list(map(
        lambda x: model(x, max_new_tokens=10)[0]['generated_text'][-1]['content'],
        tqdm(dataset['message_for_llm'])
    ))
    true = dataset['category']

    # Обработка предсказаний и истинных значений
    if normalize == True:
      print("Normalize by Levenstein distance")
      pred = list(map(
        lambda it: process.extractOne(it, list_of_categories, scorer=fuzz.token_sort_ratio)[0],
        tqdm(pred)
      ))
    elif cut_ss == True:
      print("Normalize by cutting ss")
      pred = list(map(process_text, pred))
      true = list(map(process_text, dataset['category']))
        
    return pred, true

### Оригинальные результаты на валидации

In [18]:
val_pred, val_true = get_predictions(llm_pipeline, validation_set_for_llm, normalize=True)
print(classification_report(y_true=val_true, y_pred=[x for x in val_pred]))
val_stat_df = get_predictions_stat(val_pred, val_true)
val_stat_df

  0%|          | 0/99 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Normalize by Levenstein distance


  0%|          | 0/99 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.50      0.11      0.18         9
         geography       0.67      0.25      0.36         8
            health       0.75      0.27      0.40        11
          politics       0.92      0.79      0.85        14
science/technology       0.51      0.96      0.67        25
            sports       0.52      1.00      0.69        12
            travel       0.88      0.35      0.50        20

          accuracy                           0.61        99
         macro avg       0.68      0.53      0.52        99
      weighted avg       0.68      0.61      0.56        99

All predicted and real categories:
['politics', 'science/technology', 'sports', 'health', 'travel', 'geography', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,politics,12,14,11,91.666667
1,science/technology,47,25,24,51.063830
2,sports,23,12,12,52.173913
3,health,4,11,3,75.000000
4,travel,8,20,7,87.500000
5,geography,3,8,2,66.666667
6,entertainment,2,9,1,50.000000


### Оригинальные результаты на тесте

In [19]:
test_pred, test_true = get_predictions(llm_pipeline, test_set_for_llm, normalize=True)
print(classification_report(y_true=test_true, y_pred=[x for x in test_pred]))
test_stat_df = get_predictions_stat(test_pred, test_true)
test_stat_df

  0%|          | 0/204 [00:00<?, ?it/s]

Normalize by Levenstein distance


  0%|          | 0/204 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

     entertainment       0.00      0.00      0.00        19
         geography       0.57      0.47      0.52        17
            health       0.73      0.36      0.48        22
          politics       0.90      0.60      0.72        30
science/technology       0.48      0.96      0.64        51
            sports       0.51      0.84      0.64        25
            travel       0.88      0.35      0.50        40

          accuracy                           0.58       204
         macro avg       0.58      0.51      0.50       204
      weighted avg       0.61      0.58      0.54       204

All predicted and real categories:
['politics', 'science/technology', 'sports', 'health', 'travel', 'geography', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,politics,20,30,18,90.000000
1,science/technology,102,51,49,48.039216
2,sports,41,25,21,51.219512
3,health,11,22,8,72.727273
4,travel,16,40,14,87.500000
5,geography,14,17,8,57.142857
6,entertainment,0,19,0,0.000000


### Теперь обновим функцию генерации промптов

In [20]:
def new_prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> dict:
    """
    Функция добавляет к текстам prompt с инструкцией для LLM.

    """
    assert len(categories) >= 2, f'Expected 2 or more categories, got {len(categories)} ones.'
    categories_as_string = ', '.join(categories[:-1]) + ' и ' + categories[-1]
    base_prompt = (
        f'Прочтите следующий текст и определите, какая из перечисленных тем наиболее соответствует его содержанию. '
        f'Выберите только одну наиболее подходящую тему из списка и напишите только её название, ничего больше.\n'
        f'Список тем: {categories_as_string}.\nТекст: '
    )

    system_message = {
        'role': 'system',
        'content': 'Вы - полезный помощник, который глубоко анализирует тексты на русском языке и точно определяет основную тему.'
    }

    def create_message(text_item):
        prompt = f"{base_prompt}{' '.join(text_item.split())}\nВаш ответ: "
        return [
            system_message,
            {'role': 'user', 'content': prompt}
        ]
        
    if isinstance(text, str):
        messages = create_message(text)
    else:
        messages = [create_message(it) for it in text]

    return {'message_for_llm': messages}

### И снова посмотрим на результате на валидации и на тесте

In [63]:
new_validation_set_for_llm = validation_set.map(lambda it: new_prepare_message_for_llm(it['text'], list_of_categories))
new_test_set_for_llm = test_set.map(lambda it: new_prepare_message_for_llm(it['text'], list_of_categories))
print(new_validation_set_for_llm)
print(new_test_set_for_llm['message_for_llm'][0])

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})
[{'content': 'Вы - полезный помощник, который глубоко анализирует тексты на русском языке и точно определяет основную тему.', 'role': 'system'}, {'content': 'Прочтите следующий текст и определите, какая из перечисленных тем наиболее соответствует его содержанию. Выберите только одну наиболее подходящую тему из списка и напишите только её название, ничего больше.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.\nВаш ответ: ', 'role': 'user'}]


In [64]:
val_pred, val_true = get_predictions(llm_pipeline, new_validation_set_for_llm, cut_ss=True)
print(classification_report(y_true=val_true, y_pred=[x for x in val_pred]))
val_stat_df = get_predictions_stat(val_pred, val_true)
val_stat_df

  0%|          | 0/99 [00:00<?, ?it/s]

Normalize by cutting ss
                    precision    recall  f1-score   support

     entertainment       0.50      0.11      0.18         9
         geography       0.40      0.75      0.52         8
            health       0.75      0.55      0.63        11
           politic       0.92      0.79      0.85        14
science/technology       0.71      0.88      0.79        25
             sport       0.41      1.00      0.59        12
    transportation       0.00      0.00      0.00         0
            travel       1.00      0.05      0.10        20

          accuracy                           0.60        99
         macro avg       0.59      0.52      0.46        99
      weighted avg       0.72      0.60      0.54        99

All predicted and real categories:
['science/technology', 'sport', 'transportation', 'health', 'travel', 'geography', 'politic', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,science/technology,31,25,22,70.967742
1,sport,29,12,12,41.379310
2,transportation,1,0,0,0.000000
3,health,8,11,6,75.000000
4,travel,1,20,1,100.000000
5,geography,15,8,6,40.000000
6,politic,12,14,11,91.666667
7,entertainment,2,9,1,50.000000


In [65]:
test_pred, test_true = get_predictions(llm_pipeline, new_test_set_for_llm, cut_ss=True)
print(classification_report(y_true=test_true, y_pred=[x for x in test_pred]))
test_stat_df = get_predictions_stat(test_pred, test_true)
test_stat_df

  0%|          | 0/204 [00:00<?, ?it/s]

Normalize by cutting ss
                    precision    recall  f1-score   support

     entertainment       1.00      0.11      0.19        19
         geography       0.38      0.82      0.52        17
            health       0.64      0.73      0.68        22
             music       0.00      0.00      0.00         0
           politic       1.00      0.50      0.67        30
science/technology       0.62      0.88      0.73        51
             sport       0.58      0.88      0.70        25
           theatre       0.00      0.00      0.00         0
    transportation       0.00      0.00      0.00         0
            travel       1.00      0.25      0.40        40

          accuracy                           0.61       204
         macro avg       0.52      0.42      0.39       204
      weighted avg       0.76      0.61      0.58       204

All predicted and real categories:
['science/technology', 'sport', 'transportation', 'theatre', 'music', 'health', 'travel', 'geograp

,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,science/technology,72,51,45,62.500000
1,sport,38,25,22,57.894737
2,transportation,3,0,0,0.000000
3,theatre,1,0,0,0.000000
4,music,1,0,0,0.000000
5,health,25,22,16,64.000000
6,travel,10,40,10,100.000000
7,geography,37,17,14,37.837838
8,politic,15,30,15,100.000000
9,entertainment,2,19,2,100.000000


### Также есть гипотеза, что несуществующие категории можно просто замаппить на существующие, мы енкодером получим эмбеддинги категорий и через knn, с n=1 преобразуем несуществующие категории в сущестующие.

Для проверки гипотезы отмаппим категории на словарях и посмотрим на изменения в точности.

In [69]:
def map_categories(category_mapping, predicted_categories):
    result = []
    for prediction in predicted_categories:
        if prediction in category_mapping:
            result.append(category_mapping[prediction])
        else:
            result.append(prediction)
    return result


category_mapping_dict = {
    'film': 'entertainment',
    'music': 'entertainment',
    'transportation': 'travel',
    'weather': 'geography',
    'weather/weather phenomena': 'geography',
    'the most appropriate theme for this text is "rel': 'geography',
    'theatre': 'entertainment'
}

### Валидация:

In [70]:
mapped_val_pred = map_categories(category_mapping_dict, val_pred)
print(classification_report(y_true=val_true, y_pred=[x for x in mapped_val_pred]))
mapped_val_stat_df = get_predictions_stat(mapped_val_pred, val_true)
mapped_val_stat_df

                    precision    recall  f1-score   support

     entertainment       0.50      0.11      0.18         9
         geography       0.40      0.75      0.52         8
            health       0.75      0.55      0.63        11
           politic       0.92      0.79      0.85        14
science/technology       0.71      0.88      0.79        25
             sport       0.41      1.00      0.59        12
            travel       1.00      0.10      0.18        20

          accuracy                           0.61        99
         macro avg       0.67      0.60      0.53        99
      weighted avg       0.72      0.61      0.55        99

All predicted and real categories:
['science/technology', 'sport', 'health', 'travel', 'geography', 'politic', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,science/technology,31,25,22,70.967742
1,sport,29,12,12,41.379310
2,health,8,11,6,75.000000
3,travel,2,20,2,100.000000
4,geography,15,8,6,40.000000
5,politic,12,14,11,91.666667
6,entertainment,2,9,1,50.000000


### Тест

In [71]:
mapped_test_pred = map_categories(category_mapping_dict, test_pred)
print(classification_report(y_true=test_true, y_pred=[x for x in mapped_test_pred]))
mapped_test_stat_df = get_predictions_stat(mapped_test_pred, test_true)
mapped_test_stat_df

                    precision    recall  f1-score   support

     entertainment       1.00      0.21      0.35        19
         geography       0.38      0.82      0.52        17
            health       0.64      0.73      0.68        22
           politic       1.00      0.50      0.67        30
science/technology       0.62      0.88      0.73        51
             sport       0.58      0.88      0.70        25
            travel       0.92      0.30      0.45        40

          accuracy                           0.63       204
         macro avg       0.74      0.62      0.59       204
      weighted avg       0.75      0.63      0.60       204

All predicted and real categories:
['science/technology', 'sport', 'health', 'travel', 'geography', 'politic', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,science/technology,72,51,45,62.500000
1,sport,38,25,22,57.894737
2,health,25,22,16,64.000000
3,travel,13,40,12,92.307692
4,geography,37,17,14,37.837838
5,politic,15,30,15,100.000000
6,entertainment,4,19,4,100.000000


### Мы действительно видим очень хорошие улучшения и теперь попробуем сделать это не "вручную" а уже с помощью энкодера

In [49]:
from datasets import load_dataset
import numpy as np

# Загрузка GloVe эмбеддингов
glove_dataset = load_dataset("karmiq/glove", split="train")
glove_model = {item['word']: np.array(item['embeddings']) for item in glove_dataset}

In [66]:
def map_predictions(preds, trues, glove_model):
    trues = ['science' if category == 'science/technology' else category for category in trues]
    preds = ['science' if category == 'science/technology' else category for category in preds]
        
    mapped_predictions = []
    true_categories = list(set(trues))
    true_vectors = np.array([glove_model[category] for category in true_categories])

    for pred in preds:
        if pred in true_categories:
            mapped_predictions.append(pred)
        else:
            # Если предсказанная категория не существует, находим ближайшую
            try:
                pred_vector = glove_model[pred].reshape(1, -1)
                # Вычисляем косинусное сходство между вектором предсказанной категории и векторами истинных категорий
                similarities = cosine_similarity(pred_vector, true_vectors)
                # Находим индекс ближайшей категории
                nearest_index = np.argmax(similarities)
                mapped_predictions.append(true_categories[nearest_index])
                print(f"Mapping {pred} to {true_categories[nearest_index]}")
            except KeyError:
                # Если слово не найдено в модели, добавляем None или другую метку
                mapped_predictions.append(None)

    return mapped_predictions, trues

In [67]:
mapped_val_pred, mapped_val_true = map_predictions(val_pred, val_true, glove_model)
print(classification_report(y_true=mapped_val_true, y_pred=[x for x in mapped_val_pred]))
mapped_val_stat_df = get_predictions_stat(mapped_val_pred, mapped_val_true)
mapped_val_stat_df

Mapping transportation to health
               precision    recall  f1-score   support

entertainment       0.50      0.11      0.18         9
    geography       0.40      0.75      0.52         8
       health       0.67      0.55      0.60        11
      politic       0.92      0.79      0.85        14
      science       0.71      0.88      0.79        25
        sport       0.41      1.00      0.59        12
       travel       1.00      0.05      0.10        20

     accuracy                           0.60        99
    macro avg       0.66      0.59      0.52        99
 weighted avg       0.71      0.60      0.53        99

All predicted and real categories:
['sport', 'health', 'travel', 'geography', 'science', 'politic', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,sport,29,12,12,41.379310
1,health,9,11,6,66.666667
2,travel,1,20,1,100.000000
3,geography,15,8,6,40.000000
4,science,31,25,22,70.967742
5,politic,12,14,11,91.666667
6,entertainment,2,9,1,50.000000


In [68]:
mapped_test_pred, mapped_test_true = map_predictions(test_pred, test_true, glove_model)
print(classification_report(y_true=mapped_test_true, y_pred=[x for x in mapped_test_pred]))
mapped_test_stat_df = get_predictions_stat(mapped_test_pred, mapped_test_true)
mapped_test_stat_df

Mapping transportation to health
Mapping transportation to health
Mapping transportation to health
Mapping theatre to entertainment
Mapping music to entertainment
               precision    recall  f1-score   support

entertainment       1.00      0.21      0.35        19
    geography       0.38      0.82      0.52        17
       health       0.57      0.73      0.64        22
      politic       1.00      0.50      0.67        30
      science       0.62      0.88      0.73        51
        sport       0.58      0.88      0.70        25
       travel       1.00      0.25      0.40        40

     accuracy                           0.62       204
    macro avg       0.74      0.61      0.57       204
 weighted avg       0.76      0.62      0.59       204

All predicted and real categories:
['sport', 'health', 'travel', 'geography', 'science', 'politic', 'entertainment']


,Category,Predicted Count,True Count,Correct Count,Accuracy (%)
0,sport,38,25,22,57.894737
1,health,28,22,16,57.142857
2,travel,10,40,10,100.000000
3,geography,37,17,14,37.837838
4,science,72,51,45,62.500000
5,politic,15,30,15,100.000000
6,entertainment,4,19,4,100.000000


### В итоге получаем результаты выше чем у оригинала, но ручной маппинг лучше ембеддингового так как transportation почему-то маппится в health, а не в travel